#### Get the maximum value from Burned Area Monthly tiles

Data source: https://lpdaac.usgs.gov/products/mcd64a1v006/


resources:
https://blog.dask.org/2019/06/20/load-image-data

https://examples.dask.org/applications/satellite-imagery-geotiff.html

In [1]:
from osgeo import gdal, osr
import os
import numpy as np
import dask.array as da
import time
import dask
import imageio
import glob
import sys
import shutil

In [2]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return sorted(FilesList),sorted(FileName)
        else:
            print("no"+ extension +"file for this directory")

In [12]:
folder_list=glob.glob('../data/Modis_Fire_Date/*/*/')
folder_list=sorted(folder_list)


In [13]:
folder_list_copy=folder_list.copy()
year_list=[]
for index,folder in enumerate(range(len(folder_list)//12)):
    try:
        year_list.append(folder_list_copy[0:12])
        del folder_list_copy[0:12]
    except:
        year_list.append(folder_list_copy)

        

In [ ]:
yearly_imgs_list = [[sorted(glob.glob(folder+'/tif/*.tif')) for folder in year] for year in year_list]


In [ ]:
yearly_imgs_list[9]

In [ ]:
yearly_imgs_list=yearly_imgs_list[9:]

In [ ]:
len(yearly_imgs_list)

In [ ]:
%%time
for index,year in enumerate(yearly_imgs_list):
    for index2 in range(268):
        image_status=False
        counter=0
        #define target year
        year_info=year[0][0].split('/')[3]
        #under defined year, we check every mounth with the same MODIS tile nane
        area_code=yearly_imgs_list[0][0][index2].split('.')[-4]
        
        target_imgs=[]
        target_imgs=glob.glob('../data/Modis_Fire_Date/{}/*/tif/*{}*'.format(year_info,area_code))
        #print(area_code)
        if len(target_imgs)==0:
            continue
        
        while image_status is False:
            try:
                array_list=[]
                for index,img in enumerate(target_imgs):
                    ds = gdal.Open(img)
                    #print(ds)
                    array=ds.ReadAsArray()
                    dask_array=da.from_array(array,chunks=len(array)/2)
                    array_list.append(dask_array)
            

                
                stack = da.stack(array_list, axis=0)
                max_array=stack.max(axis=0).compute()
                
                #print(target_imgs[0])
                year_name=target_imgs[0].split('/')[3]
                try:
                    output_folder='../data/Modis_Fire_Date/'+year_name+'/max_tif/'
                    os.mkdir(output_folder)
                except:
                    pass

                s_srs = ds.GetProjectionRef()    
                osng = osr.SpatialReference ()

                #use last img to define name of the last product
                # sample : 'h35v10'
                folder_name=img.split('/')[-1].split('.')[-4]
                
                osng.SetFromUserInput ( s_srs )    
                geo_t = ds.GetGeoTransform ()    
                x_size = ds.RasterXSize # Raster xsize
                y_size = ds.RasterYSize # Raster ysize
                mem_drv= gdal.GetDriverByName( 'MEM')
                dest = mem_drv.Create('', x_size,y_size, 1)
                dest.SetGeoTransform( geo_t )
                dest.SetProjection( osng.ExportToWkt())
                dest.GetRasterBand(1).WriteArray(max_array)
                output_max_path=output_folder+'modis_MCD64A1_max_'+folder_name+'.tif'
                gdal.Warp(output_max_path, dest, format = 'GTiff')
                max_array=None
                dest=None
                ds=None
                mem_drv=None
                image_status=True
                #print(output_max_path)

            except:            
                image_status=False
                counter+=1
                if counter==10:
                    print(target_imgs[0] + ' couldnt be calculated')
                    print("Unexpected error:", sys.exc_info()[0])
                    break
                continue
    
                

In [33]:
#####################################
#####################################

##### Removing folders

In [54]:
import shutil
rm=glob.glob('../data/Modis_Fire_Date/*/*/tif')
rm=sorted(rm)

In [55]:
rm[-1]

'../data/Modis_Fire_Date/2019/2019.12.01/tif'

In [56]:
for i in rm:
    #print(i)
    shutil.rmtree(i,ignore_errors=True)

In [32]:
#####################################
#####################################

#### Create yearly fire date data
With this script, you can create yearly data from fire date data. In below, script get same tile in each month and
get max value for each pixel. Last product is last fire date in each pixel. After create max tif file for each tile,
script create mosaic and convert it EPSG:4326. At the end of the script, we delete raw mosaic tif file(sinusaidal coordiante system), and
max_ tif file folders under year folder(2001/max_tif)

In [57]:
folder_list=glob.glob('../data/Modis_Fire_Date/*/*/')
folder_list=sorted(folder_list)
folder_list_copy=folder_list.copy()
year_list=[]

for index,folder in enumerate(range(len(folder_list)//12)):

    try:

        year_list.append(folder_list_copy[0:12])

        del folder_list_copy[0:12]

    except:

        year_list.append(folder_list_copy)        

In [59]:
yearly_imgs_list = [[sorted(glob.glob(folder+'/tif/*.tif')) for folder in year] for year in year_list]
len(yearly_imgs_list)

In [61]:
for index,year in enumerate(yearly_imgs_list):
    year_info=year[0][0].split('/')[3]
    print(year_info)

2015
2016
2017
2018
2019


In [62]:
%%time
for index,year in enumerate(yearly_imgs_list):
    for index2 in range(268):
        image_status=False
        counter=0
        #define target year
        year_info=year[0][0].split('/')[3]
        #under defined year, we check every mounth with the same MODIS tile nane
        area_code=yearly_imgs_list[0][0][index2].split('.')[-4]
        
        #target_imgs=[]
        target_imgs=glob.glob('../data/Modis_Fire_Date/{}/*/tif/*{}*'.format(year_info,area_code))
        #print(area_code)
        if len(target_imgs)==0:
            break
        
        while image_status is False:
            try:
                array_list=[]
                for index,img in enumerate(target_imgs):
                    ds = gdal.Open(img)
                    #print(ds)
                    array=ds.ReadAsArray()
                    array = np.where((array<0),-9999,(array))
                    dask_array=da.from_array(array,chunks=len(array)/2)
                    array_list.append(dask_array)
            

                
                stack = da.stack(array_list, axis=0)
                max_array=stack.max(axis=0).compute()
                max_array = np.where((max_array==-9999),-32768,(max_array))
                
                
                #print(target_imgs[0])
                year_name=target_imgs[0].split('/')[3]
                try:
                    output_folder='../data/Modis_Fire_Date/'+year_name+'/max_tif/'
                    os.mkdir(output_folder)
                except:
                    pass

                s_srs = ds.GetProjectionRef()    
                osng = osr.SpatialReference ()

                #use last img to define name of the last product
                # sample : 'h35v10'
                folder_name=img.split('/')[-1].split('.')[-4]
                
                osng.SetFromUserInput(s_srs)    
                geo_t = ds.GetGeoTransform ()    
                x_size = ds.RasterXSize # Raster xsize
                y_size = ds.RasterYSize # Raster ysize
                mem_drv= gdal.GetDriverByName( 'MEM')
                dest = mem_drv.Create('', x_size,y_size, 1,gdal.GDT_Int16)
                dest.SetGeoTransform( geo_t )
                dest.SetProjection( osng.ExportToWkt())
                dest.GetRasterBand(1).WriteArray(max_array)
                dest.GetRasterBand(1).SetNoDataValue(-32768)
                output_max_path=output_folder+'modis_MCD64A1_max_'+folder_name+'.tif'
                gdal.Warp(output_max_path, dest, format = 'GTiff',srcNodata=-32768,dstNodata=-32768)
                max_array=None
                stack=None
                dest=None
                ds=None
                mem_drv=None
                image_status=True
                #print(output_max_path)

            except:            
                image_status=False
                counter+=1
                if counter==10:
                    print(target_imgs[0] + ' couldnt be calculated')
                    print("Unexpected error:", sys.exc_info()[0])
                    break
                continue
    
    folder='../data/Modis_Fire_Date/'
    download_path=folder+'max_tif_mosaic/'
    tif_path=str(output_folder)
    tif_list=ListofExtensionAndName(tif_path,'.tif')
    txt_name=str(tif_path.split('/')[-3])+'_mosaic_list.txt'
    with open(txt_name, 'w') as f:
        for item in tif_list[0]:
            f.write("%s\n" % item)
    try:
        os.mkdir(download_path)
    except:
        pass
    mosaic_tif_name=download_path+'modis_MCD64A1_burned_area_'+str(tif_path.split('/')[-3])+'.tif'    
    # be careful when you define NODATA or DATA TYPE 
    command = "gdal_merge.py -init -32768 -a_nodata -32768 -ot Int16 -of GTiff -o %s --optfile %s" % (mosaic_tif_name, txt_name)
    # Run the command. os.system() returns value zero if the command was executed succesfully
    os.system(command)
    output_folder2='../data/Modis_Fire_Date/max_tif_mosaic_4326/'
    print(mosaic_tif_name)
    output_img_name=os.path.basename(mosaic_tif_name)
    output_img_path=output_folder2+output_img_name
    warp_command=f'gdalwarp -t_srs EPSG:4326 -dstnodata -32768.0 -r near -ot Int16 -of GTiff {mosaic_tif_name} {output_img_path}'
    os.system(warp_command)
    print(output_img_path)
    gdal.Warp(output_img_path, mosaic_tif_name, format = 'GTiff',srcSRS='+proj=sinu +R=6371007.181 +nadgrids=@null +wktext' ,dstSRS = 'EPSG:4326',srcNodata=-32768,dstNodata=-32768)
    os.remove(mosaic_tif_name)
    shutil.rmtree(output_folder,ignore_errors=True)
                

../data/Modis_Fire_Date/max_tif_mosaic/modis_MCD64A1_burned_area_2015.tif
../data/Modis_Fire_Date/max_tif_mosaic_4326/modis_MCD64A1_burned_area_2015.tif
../data/Modis_Fire_Date/max_tif_mosaic/modis_MCD64A1_burned_area_2016.tif
../data/Modis_Fire_Date/max_tif_mosaic_4326/modis_MCD64A1_burned_area_2016.tif
../data/Modis_Fire_Date/max_tif_mosaic/modis_MCD64A1_burned_area_2017.tif
../data/Modis_Fire_Date/max_tif_mosaic_4326/modis_MCD64A1_burned_area_2017.tif
../data/Modis_Fire_Date/max_tif_mosaic/modis_MCD64A1_burned_area_2018.tif
../data/Modis_Fire_Date/max_tif_mosaic_4326/modis_MCD64A1_burned_area_2018.tif
../data/Modis_Fire_Date/max_tif_mosaic/modis_MCD64A1_burned_area_2019.tif
../data/Modis_Fire_Date/max_tif_mosaic_4326/modis_MCD64A1_burned_area_2019.tif
CPU times: user 33min 31s, sys: 4min 4s, total: 37min 35s
Wall time: 2h 20min 19s
